<a href="https://colab.research.google.com/github/EthanXC/RedditProject/blob/main/Copy_of_RedditProject1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import pandas as pd

In [37]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [38]:
from datasets import load_dataset

In [39]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [40]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [41]:
it = iter(dataset['train'])


In [42]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])
#testing if we can get a sample comment

isn ' t this against the first amendment ? doesn ' t the first amendment give us the right to assemble and protest ?


In [43]:
data = {
    'column1_name': ['text'],
    'column2_name': ['score'],
    'column3_name': ['labels']
}

df = pd.DataFrame(data)

In [44]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['text', 'score', 'labels'])

nsfw_keywords = ['nsfw', 'adult', 'sex', 'porn', 'xxx', 'gonewild', 'nudes', 'slut', 'WTF',
'hentai', 'cum', 'rule34', 'pussy', 'onlyfans', 'milf', 'nude', 'boob']

mask = -df['labels'].str.contains('|'.join(nsfw_keywords))
df = df[mask]

for i in range(10000):  # only 10,000 rows
    extract_data = next(it)
    text = extract_data['body']
    score = extract_data['score']
    labels = extract_data['subreddit']

    new_row_data = {'text': text, "score": score, "labels": labels}
    df.loc[len(df)] = new_row_data

    if i % 1000 == 0:
        print(i)
        print(new_row_data)





0
{'text': 'oil .', 'score': -4, 'labels': 'AskReddit'}
1000
{'text': 'is that a credible list ? how did they obtain their data ?', 'score': 1, 'labels': 'starcraft'}
2000
{'text': 'ok then : would you rather live in a street full of mps , or a street full of al - qaeda members , since this video seems to be implying that the threat level is the same .', 'score': -2, 'labels': 'unitedkingdom'}
3000
{'text': 'r / bad _ cop _ no _ latte coming soon .', 'score': 24, 'labels': 'Bad_Cop_No_Donut'}
4000
{'text': 'amen', 'score': 3, 'labels': 'treecomics'}
5000
{'text': "i remember that . . . it was a form message , wasn ' t it ? i used to end every one of my gm conversations , when they asked me ' is there anything else i can do to help you ' , with something along the lines of ' i know this is a shot in the dark , but could you possibly tell me an email address with which i might submit feedback and them promptly close the ticket , disallowing further communication with you ? ' i figured i 

In [45]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [46]:
id2label = dict(enumerate(set(df['labels'])))

In [47]:
id2label

{0: 'EarthPorn',
 1: 'tf2trade',
 2: 'hiphop',
 3: 'mw3',
 4: 'Metal',
 5: 'Python',
 6: 'ferrets',
 7: 'ipad',
 8: 'bestof',
 9: 'firefly',
 10: 'Boxer',
 11: 'MinecraftCirclejerk',
 12: 'UFOs',
 13: 'picturesofiansleeping',
 14: 'obama',
 15: 'ForeverAloneDating',
 16: 'PHXMeetup',
 17: 'mylittlepony',
 18: 'hugeboobs',
 19: 'nba',
 20: 'bleach',
 21: 'unt',
 22: 'nostalgia',
 23: 'eagles',
 24: 'Hunting',
 25: 'Brooklyn',
 26: 'teenagers',
 27: 'CollegeBasketball',
 28: 'TrueReddit',
 29: 'Songwriters',
 30: 'secretsanta',
 31: 'mylittlehuman',
 32: 'Design',
 33: 'toycameras',
 34: 'ancientworldproblems',
 35: 'television',
 36: 'seduction',
 37: 'starcraft2_class',
 38: 'OFWGKTA',
 39: 'fibro',
 40: 'Millersville',
 41: 'TreesFortress2',
 42: 'jobs',
 43: 'nursing',
 44: 'ServerPorn',
 45: 'askseddit',
 46: 'Entrepreneur',
 47: 'xbox360',
 48: 'ipv6',
 49: 'MLPLounge',
 50: 'nintendo',
 51: 'askscience',
 52: 'arresteddevelopment',
 53: 'OttawaSenators',
 54: 'StAugustine',
 55: '

In [48]:
df.head()


,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [49]:
from transformers import AutoTokenizer, RobertaForSequenceClassification
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load roberta model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels = len(id2label))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,text,score,labels


In [51]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,text,score,labels


In [52]:
if np.isinf(df['score']).any():
  print("HELLO")

In [53]:
print(df.dtypes)

text      object
score      int64
labels    object
dtype: object


In [54]:
df.head()

,text,score,labels
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [55]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['text'], df['labels'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [56]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [57]:
id2label

{0: 'EarthPorn',
 1: 'tf2trade',
 2: 'hiphop',
 3: 'mw3',
 4: 'Metal',
 5: 'Python',
 6: 'ferrets',
 7: 'ipad',
 8: 'bestof',
 9: 'firefly',
 10: 'Boxer',
 11: 'MinecraftCirclejerk',
 12: 'UFOs',
 13: 'picturesofiansleeping',
 14: 'obama',
 15: 'ForeverAloneDating',
 16: 'PHXMeetup',
 17: 'mylittlepony',
 18: 'hugeboobs',
 19: 'nba',
 20: 'bleach',
 21: 'unt',
 22: 'nostalgia',
 23: 'eagles',
 24: 'Hunting',
 25: 'Brooklyn',
 26: 'teenagers',
 27: 'CollegeBasketball',
 28: 'TrueReddit',
 29: 'Songwriters',
 30: 'secretsanta',
 31: 'mylittlehuman',
 32: 'Design',
 33: 'toycameras',
 34: 'ancientworldproblems',
 35: 'television',
 36: 'seduction',
 37: 'starcraft2_class',
 38: 'OFWGKTA',
 39: 'fibro',
 40: 'Millersville',
 41: 'TreesFortress2',
 42: 'jobs',
 43: 'nursing',
 44: 'ServerPorn',
 45: 'askseddit',
 46: 'Entrepreneur',
 47: 'xbox360',
 48: 'ipv6',
 49: 'MLPLounge',
 50: 'nintendo',
 51: 'askscience',
 52: 'arresteddevelopment',
 53: 'OttawaSenators',
 54: 'StAugustine',
 55: '

In [58]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [59]:
id2label_reversed #word:number

{'EarthPorn': 0,
 'tf2trade': 1,
 'hiphop': 2,
 'mw3': 3,
 'Metal': 4,
 'Python': 5,
 'ferrets': 6,
 'ipad': 7,
 'bestof': 8,
 'firefly': 9,
 'Boxer': 10,
 'MinecraftCirclejerk': 11,
 'UFOs': 12,
 'picturesofiansleeping': 13,
 'obama': 14,
 'ForeverAloneDating': 15,
 'PHXMeetup': 16,
 'mylittlepony': 17,
 'hugeboobs': 18,
 'nba': 19,
 'bleach': 20,
 'unt': 21,
 'nostalgia': 22,
 'eagles': 23,
 'Hunting': 24,
 'Brooklyn': 25,
 'teenagers': 26,
 'CollegeBasketball': 27,
 'TrueReddit': 28,
 'Songwriters': 29,
 'secretsanta': 30,
 'mylittlehuman': 31,
 'Design': 32,
 'toycameras': 33,
 'ancientworldproblems': 34,
 'television': 35,
 'seduction': 36,
 'starcraft2_class': 37,
 'OFWGKTA': 38,
 'fibro': 39,
 'Millersville': 40,
 'TreesFortress2': 41,
 'jobs': 42,
 'nursing': 43,
 'ServerPorn': 44,
 'askseddit': 45,
 'Entrepreneur': 46,
 'xbox360': 47,
 'ipv6': 48,
 'MLPLounge': 49,
 'nintendo': 50,
 'askscience': 51,
 'arresteddevelopment': 52,
 'OttawaSenators': 53,
 'StAugustine': 54,
 'phot

In [60]:
import torch as pt

class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            print(labels)
            labels_converted = id2label_reversed[labels]
            print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}

In [61]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [62]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [63]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [64]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [65]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [66]:
%set_env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [67]:
%env CUDA_LAUNCH_BLOCKING

'1'

In [68]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

7900


In [69]:
#set(id2label_reversed.values)

In [ ]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
AskReddit
265
AskReddit
265
gaming
396
politics
177
AskReddit
265
funny
145
techsupport
664
skyrim
106
AskReddit
265
fffffffuuuuuuuuuuuu
854
guns
453
applehelp
736
tea
93
funny
145
TheRabbitHole
547
pics
266


<ipython-input-60-df3c2ef05379>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,6.810200
20,6.787000
30,6.799700
40,6.753500
50,6.765500
60,6.776400
70,6.728000
80,6.729100
90,6.539900


AskReddit
265
travel
744
wicked_edge
377
politics
177
gaming
396
fffffffuuuuuuuuuuuu
854
pittsburgh
155
politics
177
lgbt
849
MensRights
740
IAmA
76
leagueoflegends
231
starcraft
314
pics
266
tf2trade
1
AskReddit
265
guns
453
ShitRedditSays
58
AdviceAnimals
804
Rateme
132
AdviceAnimals
804
pokemon
370
pics
266
pics
266
protools
515
TwoXChromosomes
584
Paleo
405
carporn
851
buildapc
693
pics
266
AskReddit
265
politics
177
politics
177
nosleep
618
languagelearning
321
WTF
836
OperationPullRyan
335
skeptic
818
pics
266
atheism
115
funny
145
politics
177
soccer
419
hardware
472
leagueoflegends
231
SteamGameSwap
315
politics
177
AskReddit
265
SteamGameSwap
315
atheism
115
sex
641
fitnesscirclejerk
180
food
880
PHP
491
skyrim
106
neopets
503
AdviceAnimals
804
AskReddit
265
pics
266
askscience
51
pics
266
gaming
396
politics
177
AskReddit
265
WTF
836
AskReddit
265
AskReddit
265
anime
601
trees
63
AskReddit
265
trees
63
photography
55
Austin
194
BuyItForLife
143
pinkfloyd
253
politics
177
cree

In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "I want to download minecraft mods"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")




In [ ]:
outputs.logits.shape

In [ ]:
len(id2label)

In [ ]:
probabilities = torch.softmax(outputs.logits[0], dim = 0 )
print(probabilities)

In [ ]:
sort_prob = probabilities.sort(descending = True)

In [ ]:
print(sort_prob)

In [ ]:
def print_out_top_x_probs(probs, x,phrase):
  msg = phrase
  msg = "I want to download minecraft mods"
  print('Here are the top 10 most subreddits with similar')
  msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")
  probabilities = torch.softmax(outputs.logits[0], dim = 0 )
  sort_prob = probabilities.sort(descending = True)

  outputs = model(**msg_encodings)
  for n in range(x):
    sort_probs = probs
    tensor_probs = sort_probs.values[n]
    index = sort_probs.indices[n]
    sort_prob_index_int = index.item()
    subreddit_string = id2label[sort_prob_index_int]
    print(f'{n+1}. {subreddit_string} ({tensor_probs * 100}%)')

In [ ]:
print_out_top_x_probs(sort_prob, 10)

In [ ]:
phrase = input("Enter the phrase to predict the subreddit")
print_out_top_x_probs(phrase)

In [ ]:
# it's time to graph!

from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)
    return {
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy
    }


In [ ]:
import matplotlib.pyplot as plt

# this is fake data, replace with real data
epochs = 1
loss = [0.30, 0.24, 0.20]
f1_scores = [0.82, 0.85, 0.87]

plt.figure(figsize=(12, 5))

# Plotting loss
plt.subplot(1, 2, 1)
plt.plot(epochs, loss, label='Training Loss', marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plotting F1 Score
plt.subplot(1, 2, 2)
plt.plot(epochs, f1_scores, label='F1 Score', marker='o', color='r')
plt.title('F1 Score per Epoch')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()
